In [1]:
!python -V

Python 3.9.12


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
df = pd.read_parquet('./data/fhv_tripdata_2021-01.parquet')

In [7]:
len(df)

1154112

Total number of records in January:  1154112

In [8]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [9]:
df.duration.mean()

19.167224093791006

Average Duration of Trip in minutes: 19.16

In [10]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [11]:
1154112-len(df)

44286

Number of records dropped: 44286

In [12]:
categorical = ['PUlocationID', 'DOlocationID']

df[categorical] = df[categorical].fillna("-1")
df[categorical] = df[categorical].astype(str)

In [13]:
len(df[df['PUlocationID']=="-1"])/len(df)*100

83.52732770722618

Fraction of missing values for the pickup location ID: 83.5%

In [14]:
train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train.shape

(1109826, 525)

Dimension of the Matrix: 525

In [15]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.528519389548583

RMSE on train: 10.52

In [16]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.dropOff_datetime = pd.to_datetime(df.dropOff_datetime)
        df.pickup_datetime = pd.to_datetime(df.pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PUlocationID', 'DOlocationID']
    df[categorical] = df[categorical].fillna("-1")
    df[categorical] = df[categorical].astype(str)
    
    return df

In [17]:
df_val = read_dataframe('./data/fhv_tripdata_2021-02.parquet')

In [18]:
len(df_val)

990113

In [19]:
categorical = ['PUlocationID', 'DOlocationID']

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [20]:
target = 'duration'
y_val = df_val[target].values

In [21]:
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

11.014286585021035

RMSE on validation:11.01

##### Homework answers

Q1. Total number of records in January:  1154112

Q2. Average Duration of Trip in minutes: 19.16

Q3. Fraction of missing values: 83.5

Q4. Dimensionality after OHE: 525

Q5. RMSE on train: 10.52

Q6. RMSE on validation:11.01